# 데이터 가져오기 
- bike shring demand : ****https://www.kaggle.com/c/bike-sharing-demand 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import warnings
warnings.simplefilter(action='ignore')

In [ ]:
train=pd.read_csv(r'../input/bike-sharing-demand/train.csv')
test=pd.read_csv(r'../input/bike-sharing-demand/test.csv')
df=train.copy()
test_df=test.copy()
df.head()

In [ ]:
df.info()

null 인 컬럼은 없고, 

In [ ]:
test_df.info()

null 인 컬럼은 없고, 
train과 비교했을 때, 'casual','registered', 'count'가 없다.  

# 데이터 전처리

In [ ]:
# 날짜 데이터 처리 
df['datetime'] = pd.to_datetime(df['datetime'])
test_df['datetime'] = pd.to_datetime(test_df['datetime'])

df['year'] = df['datetime'].apply(lambda x: x.year)
df['month'] = df['datetime'].apply(lambda x: x.month)
df['day'] = df['datetime'].apply(lambda x: x.day)
df['hour'] = df['datetime'].apply(lambda x: x.hour)

test_df['year'] = test_df['datetime'].apply(lambda x: x.year)
test_df['month'] = test_df['datetime'].apply(lambda x: x.month)
test_df['day'] = test_df['datetime'].apply(lambda x: x.day)
test_df['hour'] = test_df['datetime'].apply(lambda x: x.hour)

df.drop(['datetime', 'casual', 'registered'], axis=1, inplace=True)
test_df.drop(['datetime'], axis=1, inplace=True)
df

In [ ]:
# 판다스의 get_dummies()를 이용해 이러한 year 칼럼을 비롯해 month, day, hour, holiday, workingday, season, weather 칼럼도 모두 원-핫 인코딩한 후에 다시 예측 성능을 확인해 보겠습니다.
df = pd.get_dummies(df, columns=['year', 'month', 'day', 'hour', 'holiday', 'workingday', 'season', 'weather'])
test_df = pd.get_dummies(test_df, columns=['year', 'month', 'day', 'hour', 'holiday', 'workingday', 'season', 'weather'])

df, test_df = df.align(test_df, join='left', axis=1)
test_df = test_df.drop(['count'], axis=1)

print(df.shape)
print(test_df.shape)

# 성능 평가 함수 정의

In [ ]:
def rmsle(y, pred):
    log_y = np.log1p(y)
    log_pred = np.log1p(pred)
    squared_error = (log_y - log_pred)**2
    rmsle = np.sqrt(np.mean(squared_error))
    return rmsle

참고 : https://www.kaggle.com/dogdriip/bike-sharing-demand#%EB%AA%A8%EB%8D%B8-%EC%84%A0%EC%A0%95 

In [ ]:
# https://www.kaggle.com/janged/xgb-lgb 
from sklearn.model_selection import GridSearchCV

df_train_target = df['count']
df_train_features = df.drop('count',axis=1)


def print_best_params(model, params):
    grid_model = GridSearchCV(
        model, 
        param_grid = params,
        cv=5,
        scoring='neg_mean_squared_error')

    grid_model.fit(df_train_features, df_train_target)
    rmse = np.sqrt(-1*grid_model.best_score_)
    print(
        '{0} 5 CV 시 최적 평균 RMSE 값 {1}, 최적 alpha:{2}'.format(model.__class__.__name__, np.round(rmse, 4), grid_model.best_params_))
    return grid_model.best_estimator_

In [ ]:
df['count'] = np.log1p(df['count'])

# 모델 테스트

In [ ]:
# 선형 회귀 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso


# xgb_model = XGBRegressor(n_estimators=500)

x_train,x_test,y_train,y_test=train_test_split(df.drop('count',axis=1),df['count'],test_size=0.3,random_state=42)

# test_df.head()
# xgb_model.fit(x1_train, y1_train)
# xgb_pred = xgb_model.predict(x_test)

# y_test_exp = np.expm1(y_test)
# pred_exp = np.expm1(xgb_pred)
# print('RMSLE:', rmsle(y_test_exp, pred_exp))

lr_reg = LinearRegression()
lr_reg.fit(x_train, y_train)
lr_pred = lr_reg.predict(x_test)


y_test_exp = np.expm1(y_test)
lr_pred_exp = np.expm1(lr_pred)
print('LinearRegression RMSLE:', rmsle(y_test_exp, lr_pred_exp))


In [ ]:
# RandomForest 
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(x_train, y_train)
rf_pred = rf_model.predict(x_test)

y_test_exp = np.expm1(y_test)
rf_pred_exp = np.expm1(rf_pred)
print('RandomForest RMSLE:', rmsle(y_test_exp, rf_pred_exp))

In [ ]:
# xgboost 
from xgboost import XGBRegressor

# xgb_model = XGBRegressor() 
# xgb_model = XGBRegressor(n_estimators=500) # n_estimators : 반복하는 트리 갯수 (크면 성능 올라가지만 너무 크면 과적합)
xgb_model = XGBRegressor(learning_rate=0.2) # learning_rate

xgb_model.fit(x_train, y_train)
xgb_pred = xgb_model.predict(x_test)


y_test_exp = np.expm1(y_test)
xgb_pred_exp = np.expm1(xgb_pred)
print('xgboost RMSLE:', rmsle(y_test_exp, xgb_pred_exp))

In [ ]:
# LightGBM - https://www.youtube.com/watch?v=1dSfWpFnvP0 
# - 학습과 예측 속도 빠름. 카테고리형 피쳐 자동 변환
# 트리분할 : 리프 중심(Leaf-wise) 트리 분할 방식(비대칭적 규칙 트리)
#  - 장점 : 예측 오류 손실 최소화 / 단점 : 오버 피팅에 약함 
from lightgbm import LGBMRegressor

lgb_params = {
#     'objective':['regression'],
#     'num_leave' : [1],
    'learning_rate' : [0.05],
    'n_estimators':[500],
    'max_bin' : [80],
#     'gpu_id':[0] ,         
#     'tree_method':['gpu_hist'],
#     'predictor':['gpu_predictor'],
#     'refit':[True]
}

lgb_model = LGBMRegressor()


lgb_model.fit(x_train, y_train)
lgb_pred = lgb_model.predict(x_test)
y_test_exp = np.expm1(y_test)
lgb_pred_exp = np.expm1(lgb_pred)
print('LGBMRegressor RMSLE:', rmsle(y_test_exp,lgb_pred_exp))

lgb_estimator = print_best_params(lgb_model, lgb_params)
# lgb_best_param_pred = lgb_estimator.predict(x_test)
# lgb_best_param_pred_exp = np.expm1(lgb_best_param_pred)
# print('LGBMRegressor with best Param RMSLE:', rmsle(y_test_exp,lgb_best_param_pred_exp))

# 제출

In [ ]:
X_train = df.drop(['count'], axis=1)
y_train = df['count']
X_test = test_df

X_test

In [ ]:
lgb_model = LGBMRegressor()
lgb_model.fit(X_train, y_train)
pred = lgb_model.predict(X_test)
pred_exp = np.expm1(pred)

submission = pd.read_csv('../input/bike-sharing-demand/sampleSubmission.csv')
submission.loc[:, 'count'] = pred_exp
submission

In [ ]:
submission.to_csv('submission.csv', index=False)